# 1. Load Data

#### 1.1 Import images

In [ ]:
from scripts.deepgtav.messages import frame2numpy
import numpy as np
import pickle
import bunch
import gzip
import os
import cv2

%matplotlib inline
import matplotlib.pyplot as plt

# open the file
filepath = os.path.join(os.getcwd(), 'data')
filename = "dataset_mini.pz"
pfile = gzip.open(os.path.join(filepath, filename), mode='rb')

#### 1.2 PCA Function for dimensionality reduction on images

In [ ]:
def apply_pca(image, dim=10):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = np.matrix(image)
    U, S, V = np.linalg.svd(image)
    reconst = np.matrix(U[:, :dim]) * np.diag(S[:dim]) * np.matrix(V[:dim, :])
    reconst = reconst[...,None]
    return reconst

#### 1.3 Setting extracted data to specific variables

In [ ]:
count = 0
no_of_channels = 3

frames_per_weather = 20000
reps_per_weather = 8
frames_per_rep = int(frames_per_weather/reps_per_weather)
image_size = (100, 75)     # required image size
minimap_size = (170, 110)  # current minimaps size

images = []
minimaps = []
outputs = []
speeds = []
yawRates = []

# find resize ratio
diff = []
diff.append(minimap_size[0] - image_size[0])
diff.append(minimap_size[1] - image_size[1])
ratio = 1- (np.min(diff) / minimap_size[np.argmin(diff)])

while True:
    try:
        
        # Load var from pickle
        var = pickle.load(pfile)
        
        # Skip first n frames
        count += 1
        # if count<3000: continue
            
        # Save image, minmap and steering
        image = var['frame']
        image = cv2.resize(image, (200,150))
        image = apply_pca(image, dim=20)
        images.append(image)
        
        minimap = var['map']
        minimap = cv2.resize(minimap, (100,75))
        minimap = minimap[:,:,:]
        minimaps.append(minimap)
        
        speeds.append(var['speed'])
        yawRates.append(var['yawRate'])
        outputs.append([var['steering'], var['throttle'], var['brake']])
        # outputs.append([var['steering']])
        
        # Display image
        if count % 100 == 0:
            plt.subplot(121); plt.imshow(image[:,:,0]); plt.axis('off')
            plt.subplot(122); plt.imshow(minimap[:,:,::-1]); plt.axis('off')
            plt.show()
        
        if count == 1: print([image.shape, minimap.shape]); print(var.keys())
        if count>5000: break
            
    except EOFError: break

#### 1.4 Clipping steering values

In [ ]:
print(np.min(y), np.max(y))
np.clip(y, -1, 1, out=y)
print(np.min(y), np.max(y))

#### 1.5 Finding previous steering values

In [ ]:
pick_last_steering = 8
holder = np.zeros((y.shape[0], pick_last_steering))
gamma = 0.8
for i in range(holder.shape[0]):
    count = 0
    for j in range(i, i-pick_last_steering, -1):
        holder[i][count] = y[j-pick_last_steering]*(j>0)#*(gamma**(j-pick_last_steering))
        count += 1
print(holder.shape)

In [ ]:
images = np.array(images)
minimaps = np.array(minimaps)
speeds = np.array(speeds)
yawRates = np.array(yawRates)
holder = np.array(holder)

# Get inputs and outputs
# x = {}
# x.images = images
# x.minimaps = minimaps
# x = np.concatenate((images, minimaps), axis=1)
# x = [images, minimaps, speeds, yawRates, holder]
x = [images, minimaps, holder, speeds, yawRates]

x_shape = [entity.shape for entity in x]
print('Dataset Shape: x: {} | y: {}'.format(x_shape, y.shape))

# 2. Data Preprocessing

#### 2.1 Normalise images to [-1, 1]

In [ ]:
print([[np.min(entity), np.max(entity)] for entity in x])
if not isinstance(x, np.ndarray): 
    x_new = [(entity/255 - 0.5) * 2 for i, entity in enumerate(x) if i<2]
    if len(x)>=2: x_new = [*x_new, *x[2:]]
    x = x_new
[[np.min(entity), np.max(entity)] for entity in x]

#### 2.2 Data splitting

In [ ]:
seed = 1
np.random.seed(seed)

recv = [None for i in range(2*len(x))]
from sklearn.model_selection import train_test_split
*recv, y_train, y_test = train_test_split(*x, y, test_size=0.2, random_state=np.random, shuffle=True)

x_train = recv[::2]
x_test = recv[1::2]

# print("Train Data | Test Data")
# x_train = [x0_train, x1_train]
# x_test = [x0_test, x1_test]
print(("{0:^10} | {1:^10}").format(x_train[0].shape[0], x_test[0].shape[0]))

# 3. Building Model

#### 3.1 Importng relevant libraries

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.layers import BatchNormalization
from keras.layers.merge import concatenate
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.metrics import mae
from keras.initializers import RandomNormal
from keras.regularizers import Regularizer
import keras.applications.inception_v3 import InceptionV3

from keras import backend
backend.set_image_dim_ordering('tf')

#### 3.2 Create reusable functions

In [ ]:
h_image = x[1].shape[1]
w_image = x[1].shape[2]
c_image = x[1].shape[3]

h_maps = x[1].shape[1]
w_maps = x[1].shape[2]
c_maps = x[1].shape[3]

no_of_classes = y.shape[0]
#print(x.shape, y.shape)

def evaluate_model(model):
    print("\n\n")
    scores = model.evaluate(x_test, y_test)
    print("Accuracy: ", scores[1]*100, "%")
    return scores

#### 3.3 Create helper functions for building model

In [ ]:
def bn_relu(image, channel):
    # BNORM -> RELU
    bnorm = BatchNormalization(channel)(image)
    activ = Activation('relu')(bnorm)
    return activ



#### 3.3 Create the ResNet